In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from lib.DrugUtil import flatten, DrugUtil

import plotly.express as px
from lib.FhirDataUtil import FhirDataUtil
from lib.CCSDataUtil import CCSDataUtil
from IPython.display import display, HTML


def printmd(string):
    display(string)

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', None)

_output_dir = 'output/'

_rxnorm_file = "input_static/RxTerms202203/RxTerms202203.txt"
_rxnorm_ingredients_file = "input_static/RxTerms202203/RxTermsIngredients202203.txt"
_rxclass_file = "input_static/_rxclass_2022-04-10.csv"

_ccs_demographics_file = "input/ccs/ccs_demographics.csv"
_ccs_medications_file = "input/ccs/ccs_medications.csv"
_ccs_conditions_file = "input/ccs/ccs_conditions.csv"

_fhir_demographics_file = "input/fhir_20230423/fhir_demographics.csv"

_pc_demographics_file = "input/pcornet/demographic_fhir.csv"

_cache_dir = 'cache/'

In [7]:
fhirUtil = FhirDataUtil()
drugUtil = DrugUtil()
drugUtil.load(rxnorm_file=_rxnorm_file,
                        rxnorm_ingredients_file=_rxnorm_ingredients_file, 
                        rxclass_file=_rxclass_file)

ccsUtil = CCSDataUtil(drugUtil)
ccsUtil.load_demographics(_ccs_demographics_file)
ccs_demo = ccsUtil.demographics
ccsUtil.load_conditions(_ccs_conditions_file)
ccs_cond = ccsUtil.conditions

fhirUtil.load_demographics(fhir_demographics_file=_fhir_demographics_file)
fhir_demo = fhirUtil.demographics

pc_demo = pd.read_csv(_pc_demographics_file)

Reading rxclass file...
Reading rxnorm file...


/app/lib/CCSDataUtil.py:75: DtypeWarning: Columns (11,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  self.demographics = pd.read_csv(ccs_demographics)
/app/lib/CCSDataUtil.py:13: DtypeWarning: Columns (5,6,11,12,13,18) have mixed types. Specify dtype option on import or set low_memory=False.
  self.conditions = pd.read_csv(ccs_conditions_file, delimiter=',')


Loaded demographics file with entries:  99064
Loaded CCS conditions records: 95301


In [3]:

ccs_demo.drop_duplicates('user_id').merge(fhir_demo.drop_duplicates('user_id'), on='user_id', how="outer", indicator=True)['_merge'].value_counts()

left_only     97508
both           1555
right_only       48
Name: _merge, dtype: int64

In [8]:

ccs_demo.drop_duplicates('user_id').merge(pc_demo.drop_duplicates('user_id'), on='user_id', how="outer", indicator=True)['_merge'].value_counts()


left_only     98436
both            627
right_only        2
Name: _merge, dtype: int64

In [10]:
fhir_demo.drop_duplicates('user_id').merge(pc_demo.drop_duplicates('user_id'), on='user_id', how="outer", indicator=True)['_merge'].value_counts()


left_only     985
both          618
right_only     11
Name: _merge, dtype: int64

In [11]:
ccs_demo['user_id'].nunique()

99063

In [4]:
chunksize = 500000

list_of_dataframes = []

for chunk in pd.read_csv(_ccs_medications_file, chunksize=chunksize, low_memory=False):
    # process your data frame here
    # then add the current data frame into the list
    print('loaded', len(list_of_dataframes)*chunksize, 'records')
    chunk = chunk[chunk['user_id'].isin(fhir_demo['user_id'])]
    list_of_dataframes.append(chunk)

# if you want all the dataframes together, here it is
ccs_meds = pd.concat(list_of_dataframes)

print("Total ccs_meds entries loaded:", len(ccs_meds))

loaded 0 records
loaded 500000 records
loaded 1000000 records
loaded 1500000 records
loaded 2000000 records
loaded 2500000 records
loaded 3000000 records
loaded 3500000 records
loaded 4000000 records
loaded 4500000 records
Total ccs_meds entries loaded: 296322


In [54]:
# Select only the FHIR users
ccs_meds = ccs_meds[ccs_meds['user_id'].isin(fhir_demo['user_id'])]
ccs_meds["submitted_at"] = pd.to_datetime(ccs_meds["submitted_at"])
print('Selected users:', ccs_meds['user_id'].nunique())
print('Total Meds:', len(ccs_meds))

Selected users: 709
Total Meds: 296322


In [55]:
# De-duplicate by rxcui
ccs_meds = ccs_meds[(~ccs_meds[['user_id', 'rxcui']].duplicated()) | ccs_meds['rxcui'].isna()]
ccs_meds["submitted_at"] = pd.to_datetime(ccs_meds["submitted_at"])
ccs_meds = ccs_meds.drop_duplicates(['user_id', 'medication_name'])
print('total meds after de-duplication:', ccs_meds.shape[0])

total meds after de-duplication: 6186


In [7]:
# Keep only latest entry for each user
# latest_date_per_user = ccs_meds.groupby('user_id')['submitted_at'].max().reset_index()
# ccs_meds = ccs_meds.merge(latest_date_per_user, on=['user_id', 'submitted_at'], how='inner')
# ccs_meds = ccs_meds.drop_duplicates(['user_id', 'rxcui'])
# print("Keeping only the latest set of entries for each user", len(ccs_meds))

In [8]:
ccs_meds['submitted_at'].sort_values()

163156    2020-01-23
19501     2020-01-23
163153    2020-01-23
163154    2020-01-23
163155    2020-01-23
             ...    
4556823   2023-05-16
2044994   2023-05-16
4129453   2023-05-16
3355537   2023-05-16
1467033   2023-05-16
Name: submitted_at, Length: 6186, dtype: datetime64[ns]

In [9]:
ccs_meds = ccs_meds.merge(drugUtil.rxnormIngr, right_on='RXCUI', left_on='rxcui', how='left', indicator='ingr_merge')
ccs_meds = ccs_meds.rename(columns={'ING_RXCUI': 'ccs_ing_code', 'INGREDIENT': 'ccs_ing_name'})
ccs_meds['ingr_merge'].value_counts()

both          5322
left_only     1246
right_only       0
Name: ingr_merge, dtype: int64

In [10]:
print('Number of custom entries:', len(ccs_meds[pd.isna(ccs_meds['ccs_ing_name'])]))

Number of custom entries: 1246


In [11]:
ccs_meds[1:3]

,user_id,site,submitted_at,custom_entry,medication_name,brand_name,full_generic_name,product_name,sxdg_name,rxcui,generic_rxcui,sxdg_rxcui,route,dosage_form,strength,currently_taking,as_needed,frequency_number,frequency_every,frequency_unit,frequency_times_per_unit,variable_basis,reason,RXCUI,ccs_ing_name,ccs_ing_code,ingr_merge
1,707,covid19,2023-04-04,False,BENADRYL (Oral Pill),BENADRYL,diphenhydramine hydrochloride 25 MG Oral Capsule,diphenhydramine hydrochloride 25 MG Oral Capsule [Benadryl],Benadryl Pill,1049910.0,1049909.0,1170149.0,Oral Pill,Oral Capsule,25 mg,True,True,0.0,0.0,hour,NaN,NaN,NaN,1049910.0,diphenhydramine,3498.0,both
2,707,covid19,2023-04-04,False,benadryl pill,BENADRYL,NaN,NaN,benadryl pill,NaN,NaN,1170149.0,Oral Pill,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,not known,allergies,NaN,NaN,NaN,left_only


In [12]:
# Here we add well-known misspellings for custom entries

ccs_meds_p = ccs_meds

mapping_dictionary = {
         'ASPIRIN': 1191,
         'asprin': 1191,
         'thyroxine': 10582, 
         'synthroid': 10582,
         'albuterol': 435, 
         'ventolin': 435,
         'vitamin d': 2418,
         'vitamin c': 1151,
         'Ethinyl estradiol/Inert ingredients/Norgestimate': [4124, 31994]
}

def add_custom_ingredients(df_meds, mapping_dictionary, 
                           search_column="medication_name",
                           ing_rxcui_column="rxcui_ing",
                           ing_name_column="ing_name", verbose=1):
    # ADDS [INGREDIENT_LIIST] AND [ING_RXCUI_LIST] COLUMNS TO DF, USING COLUMN LABELED '
    # df_meds = df with string column 'medication_name' <string> and 'custom_entry' <boolean>
    # verbose - verbose level (0 = no verbose), (1 - partial verbose), (2 - max verbose for debugging)
    for substring, ing_rxcui in mapping_dictionary.items():
        if (not isinstance(ing_rxcui, list)): ing_rxcui = [ing_rxcui]
        ing_rxcui_list = []
        ing_name_list = []
        for i in ing_rxcui:
            rxcui, ing_name = drugUtil.findIngredientByRxcui(rxcui_ing=i)
            if (ing_name == None):
                print(f'Unable to find ing_rxcui {i} in rxnormIngr')
            else:
                ing_rxcui_list.append(rxcui)
                ing_name_list.append(ing_name)
        
        if len(ing_rxcui_list) == 0: 
            print(f'NOT found {substring}, {ing_rxcui} records in ingredient database')
            continue
        
        search_filter = df_meds[search_column].str.contains(substring, na=False, case=False) & df_meds.custom_entry == True
        count = len(df_meds[search_filter])
#         if verbose: print(f'Found {count} df_meds records matching {substring}')
        df_meds.loc[search_filter, 
                    ing_name_column] = ','.join(ing_name_list)
        df_meds.loc[search_filter, 
                    ing_rxcui_column] = ','.join(map(str, ing_rxcui_list))
        if verbose > 0: display(HTML(f'Searched <b>{substring}</b> found <b>{ing_name}</b>, added ingredients to <b>{count}</b> records'))
    return df_meds

ccs_meds_p = add_custom_ingredients(ccs_meds_p, mapping_dictionary)

# Convert to array for explode
def unpack(x):
    if pd.isna(x): return x
    return x.split(',')
ccs_meds_p['ing_name'] = ccs_meds_p['ing_name'].apply(unpack)
ccs_meds_p['rxcui_ing'] = ccs_meds_p['rxcui_ing'].apply(unpack)

ccs_meds_p = ccs_meds_p.explode(['rxcui_ing', 'ing_name'])

ccs_meds_p['ccs_ing_code'] = ccs_meds_p['ccs_ing_code'].fillna(ccs_meds_p['rxcui_ing'])
ccs_meds_p['ccs_ing_name'] = ccs_meds_p['ccs_ing_name'].fillna(ccs_meds_p['ing_name'])
ccs_meds_p = ccs_meds_p.drop(['rxcui_ing', 'ing_name'], axis=1)
ccs_meds = ccs_meds_p

In [13]:
ccs_meds['idx'] = ccs_meds.index
nancodes = ccs_meds[pd.isna(ccs_meds['ccs_ing_code'])]
print('number of nancodes:', len(nancodes))

number of nancodes: 1045


In [17]:

nancodes = drugUtil.add_ingredient_columns(nancodes, 'medication_name', new_code_column='ing_code', new_name_column='ing_name', max_distance=1)
ccs_meds = ccs_meds.drop(['ing_code', 'ing_name'], axis=1) if 'ing_code' in ccs_meds else ccs_meds
ccs_meds = ccs_meds.merge(nancodes[['idx','ing_code', 'ing_name']], on='idx', how='left')
ccs_meds = ccs_meds.explode(['ing_code', 'ing_name'])
ccs_meds['ccs_ing_code'] = ccs_meds['ccs_ing_code'].fillna(ccs_meds['ing_code'])
ccs_meds['ccs_ing_name'] = ccs_meds['ccs_ing_name'].fillna(ccs_meds['ing_name'])


100 / 1045
200 / 1045
300 / 1045
400 / 1045
500 / 1045
600 / 1045
700 / 1045
800 / 1045
900 / 1045
1000 / 1045


/app/lib/DrugUtil.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[[new_name_column, new_code_column]] = df[med_name_column].apply(self.search_ingredient_by_substring, max_distance=max_distance)


In [9]:
print('remaining nan codes:', ccs_meds[pd.isna(ccs_meds['ccs_ing_code'])].shape[0])
print('total codes', len(ccs_meds))

remaining nan codes: 162
total codes 8939


In [21]:
ccs_meds.to_csv(_cache_dir+'/ccs_meds_mapped.csv')

In [3]:
ccs_meds = pd.read_csv(_cache_dir+'/ccs_meds_mapped.csv')

In [10]:
ccs_meds

,Unnamed: 0,user_id,site,submitted_at,custom_entry,medication_name,brand_name,full_generic_name,product_name,sxdg_name,rxcui,generic_rxcui,sxdg_rxcui,route,dosage_form,strength,currently_taking,as_needed,frequency_number,frequency_every,frequency_unit,frequency_times_per_unit,variable_basis,reason,RXCUI,ccs_ing_name,ccs_ing_code,ingr_merge,idx,ing_code,ing_name
0,0,707,covid19,2023-04-04,False,ADVIL (Chewable),ADVIL,ibuprofen 100 MG Chewable Tablet,ibuprofen 100 MG Chewable Tablet [Advil],Advil Chewable Product,731536.0,310963.0,1296361.0,Chewable,Chewable Tablet,100 mg,True,True,NaN,NaN,NaN,NaN,NaN,NaN,731536.0,ibuprofen,5640.0,both,0,NaN,NaN
1,1,707,covid19,2023-04-04,False,BENADRYL (Oral Pill),BENADRYL,diphenhydramine hydrochloride 25 MG Oral Capsule,diphenhydramine hydrochloride 25 MG Oral Capsule [Benadryl],Benadryl Pill,1049910.0,1049909.0,1170149.0,Oral Pill,Oral Capsule,25 mg,True,True,0.0,0.0,hour,NaN,NaN,NaN,1049910.0,diphenhydramine,3498.0,both,1,NaN,NaN
2,2,707,covid19,2023-04-04,False,benadryl pill,BENADRYL,NaN,NaN,benadryl pill,NaN,NaN,1170149.0,Oral Pill,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,not known,allergies,NaN,diphenhydramine,3498.0,left_only,2,3498.0,diphenhydramine
3,2,707,covid19,2023-04-04,False,benadryl pill,BENADRYL,NaN,NaN,benadryl pill,NaN,NaN,1170149.0,Oral Pill,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,not known,allergies,NaN,diphenhydramine,3498.0,left_only,2,58295.0,zinc acetate
4,3,707,covid19,2023-04-04,False,benadryl pill,BENADRYL,NaN,NaN,benadryl pill,NaN,NaN,1170149.0,Oral Pill,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,not known,allergies,NaN,zinc acetate,58295.0,left_only,2,3498.0,diphenhydramine
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8934,6973,585388,covid19,2023-02-08,False,PEPCID (Oral Pill),PEPCID,famotidine 20 MG Oral Tablet,famotidine 20 MG Oral Tablet [Pepcid],Pepcid Pill,104094.0,310273.0,1185207.0,Oral Pill,Oral Tablet,20 mg,True,False,60.0,12.0,hour,NaN,NaN,MCAS,104094.0,famotidine,4278.0,both,6563,NaN,NaN
8935,6974,585388,covid19,2023-02-08,False,QVAR (Inhalant),QVAR,Breath-Actuated 120 ACTUAT beclomethasone dipropionate 0.04 MG/ACTUAT Metered Dose Inhaler,Breath-Actuated 120 ACTUAT beclomethasone dipropionate 0.04 MG/ACTUAT Metered Dose Inhaler [Qvar],Qvar Inhalant Product,1998774.0,1998773.0,1177118.0,Inhalant,Metered Dose Inhaler,40 mcg/puff,True,False,2.0,12.0,hour,NaN,NaN,"Shortness of breath, MCAS, POTS, vascular inflammation",1998774.0,beclomethasone,1347.0,both,6564,NaN,NaN
8936,6975,585388,covid19,2023-02-08,False,SYNTHROID (Oral Pill),SYNTHROID,levothyroxine sodium 0.075 MG Oral Tablet,levothyroxine sodium 0.075 MG Oral Tablet [Synthroid],Synthroid Pill,966171.0,966222.0,1186074.0,Oral Pill,Oral Tablet,0.075 mg,True,False,1.0,24.0,hour,NaN,NaN,Hypothyroidism,966171.0,levothyroxine,10582.0,both,6565,NaN,NaN
8937,6976,585388,covid19,2023-02-08,False,Topiramate (Oral Pill),NaN,topiramate 50 MG Oral Tablet,topiramate 50 MG Oral Tablet,topiramate Pill,151226.0,NaN,1162750.0,Oral Pill,Oral Tablet,50 mg,True,False,3.0,24.0,hour,NaN,NaN,Migraines,151226.0,topiramate,38404.0,both,6566,NaN,NaN


In [6]:
ccs_demo_fhir = ccs_demo[ccs_demo['user_id'].isin(fhir_demo['user_id'])]
len(ccs_demo_fhir)

1555

In [7]:
ccs_demo_fhir[~ccs_demo_fhir['user_id'].isin(ccs_meds['user_id'])]['user_id'].nunique()

852

In [65]:
ccs_demo[ccs_demo['user_id']==305407]

,user_id,evisit,dt,survey_version,sex,gender,gender_identity,race,asian,pacisland,ethnicity,hispanic,intro,ladder_image,ladder,education,race-1,race-2,race-4,race-5,race-3,race-6,race-7,gender-1,gender-2,gender-3,gender-4,gender-5,gender-6,gender-7
0,305407,1,2020-03-26 04:52:59,0,1.0,1.0,NaN,2,NaN,NaN,NaN,1.0,NaN,NaN,10,9,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
len(ccs_demo[ccs_demo['user_id'].isin(ccs_cond['user_id'])].drop_duplicates('user_id'))/len(ccs_demo)

0.9555337963336833

In [3]:
len(ccs_demo[ccs_demo['user_id'].isin(ccs_cond['user_id'])].drop_duplicates('user_id'))


94659

In [59]:
ccs_demo_fhir[ccs_demo_fhir['user_id'].isin(ccs_cond['user_id'])]['user_id'].nunique()

1551

In [ ]:
ccs_meds[ccs_meds['user_id'] == 305407]